## Running FairNow's User Data Bias Testing

#### FairNow's User Data Testing is a way to evaluate a model for bias using real data. Please do not send any PII data.

In [ ]:
import os
import requests
import json
from time import sleep

### Prerequisites:

#### To use this notebook, you'll need a `client_id` and `client_secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these stored as environment variables:

* FAIRNOW_CLIENT_ID
* FAIRNOW_CLIENT_SECRET

#### To run this you will need a `model_id` and `version` for the specific model you want to test. Details of how to create and lookup models can be found here: https://github.com/FairNow/API-Guides/blob/main/notebooks/Models%20API.ipynb

#### Finally, you'll need a `threshold` value, which is the value at which anyone with a score above is considered a passing score.

In [ ]:
# Get the client secret and Id needed for OAuth2.0:
client_id = os.getenv("FAIRNOW_CLIENT_ID")
client_secret = os.getenv("FAIRNOW_CLIENT_SECRET")

model_id = "{model_id}" # Replace with the correct modelId
version = "{version}" # Replace with the correct version

threshold = {threshold value} # Replace with threshold value, a number between 0 and 1

#### You will also need to prepare a CSV file to upload containing data. The first row contains the column names.

#### The following columns are required:
* `Race`
* `Gender`
* `TimeStamp` (ISO8601 Timestamp, e.g `2023-12-14T16:26:05.898156Z`)
* `Score` (a number between 0 and 1)

#### Additional columns can be added to allow filtering of data, e.g. `Job Title`, `Location` etc

#### First, let's get an access token:

In [ ]:
access_token = None

# Call the Auth endpoint to request a token:
fairnow_token_endpoint = "https://auth.fairnow.ai/oauth2/token"
scope = "https://auth.fairnow.ai/FULL_ACCESS"

token_request_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': scope
}

try:
    response = requests.post(fairnow_token_endpoint, data=token_request_data)
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        print('Successfully created token')
    else:
        print(f'Error: {response.status_code} - {response.text}')
        print(response)


except Exception as e:
    print(f'Request failed: {e}')

In [ ]:
#### Set up headers and endpoints that we will be using:

In [ ]:
headers = {"Authorization": f"Bearer {access_token}", "Accept": "application/json"}

fairnow_api = "https://api.fairnow.ai/v1"
url = f"{fairnow_api}/userData/{model_id}/{version}"

#### Next we create a task. We need to specify the model and version, and pass the threshold value. The response will include the `taskId` and the presigned URL to upload the CSV file.

In [ ]:
payload = {
    "threshold": threshold
}

response = requests.post(url, headers=headers, json=payload)

task_id = response.json()["task"]["taskId"]
presigned_url = response.json()["task"]["uploadURL"]
key = response.json()["task"]["key"]
fields = response.json()["task"]["fields"]

#### Use the link to upload scores. Once the scores are uploaded, this triggers the analysis job. This runs in the background again and can take a few minutes.

In [ ]:
scores_path = 'scores.csv'
files = {'file': (key, open(scores_path, 'r'))}

response = requests.post(presigned_url, data=fields, files=files)

print(response)

#### We'll query the API again to know when the analysis has been finished

In [ ]:
get_task_url = f"{fairnow_api}/userData/tasks/{task_id}"

response = requests.get(get_task_url, headers=headers)
current_status = response.json()['task']['status']

while current_status != 'READY':
    print('Polling task table to learn status every 15 seconds')
    response = requests.get(get_task_url, headers=headers)
    current_status = response.json()['task']['status']
    print(f'Task status: `{current_status}`')
    print()
    sleep(15)

print(f'Analysis results ready to download.')

analysis_download_presigned_url = response.json()["task"]["presignedUrlAnalysisResults"]

#### Once the analysis task is ready, it returns a presigned link you can use to download the analysis results. The output is a csv with the results by race and gender.

In [ ]:
response = requests.get(analysis_download_presigned_url)

with open('results.csv', 'wb') as file:
    file.write(response.content)

In [ ]:
!cat results.csv